# Topographic attributes

Notebook to create the file `CAMELS_DE_topographic_attributes.csv`.  

columns in CAMELS-GB:
- gauge_id
- provider_id
- gauge_name
- water_body_name
- federal_state
- gauge_lat [°]
- gauge_lon [°]
- gauge_easting [m]
- gauge_northing [m]
- gauge_elev_metadata [m a.s.l.]
- area_metadata [km²]
- gauge_elev [m a.s.l.]
- area [km²]
- elev_mean [m a.s.l.]
- elev_min [m a.s.l.]
- elev_5 [m a.s.l.]
- elev_50 [m a.s.l.]
- elev_95 [m a.s.l.]
- elev_max [m a.s.l.]


In [1]:
from glob import glob
import pandas as pd

from camelsp import get_metadata, Station

In [2]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 1460


## Read DEM data

In [3]:
# read metadata, filter for camels_ids
meta = get_metadata()
meta = meta[meta["camels_id"].isin(camels_ids)].sort_values("camels_id").reset_index(drop=True)


dem = pd.read_csv("../../../camelsp/output_data/raw_catchment_attributes/dem/copernicus_dem30/dem_extracted.csv")
dem = dem[dem["camels_id"].isin(camels_ids)].sort_values("camels_id").reset_index(drop=True)

dem

,camels_id,gauge_elevation,gauge_elevation_from_dem,mean_catchment_elevation_from_dem,min_catchment_elevation_from_dem,max_catchment_elevation_from_dem,quantile0.05_catchment_elevation_from_dem,quantile0.5_catchment_elevation_from_dem,quantile0.95_catchment_elevation_from_dem,stdev_catchment_elevation_from_dem
0,DE110000,657.334,661.994324,822.254089,658.528503,1159.789551,681.916333,792.256984,1037.072260,113.295898
1,DE110010,649.162,650.837585,820.253540,649.943359,1159.789551,680.975647,792.741199,1033.215935,111.002815
2,DE110020,542.530,546.443787,774.901062,541.592651,1159.789551,621.953125,765.674703,962.753594,106.885468
3,DE110030,489.903,493.753967,729.470520,488.794952,1159.789551,555.287759,726.973546,933.637332,115.808411
4,DE110040,489.317,491.345367,628.930176,490.260468,791.106323,524.483190,626.595830,733.947472,64.001694
...,...,...,...,...,...,...,...,...,...,...
1455,DEG10580,233.323,236.299713,486.587280,234.860596,942.446045,293.895073,459.052026,775.268522,149.568130
1456,DEG10590,222.678,225.585602,473.413147,223.506714,988.187073,282.769750,451.887895,736.467484,132.760834
1457,DEG10600,174.317,176.605453,350.211731,176.305817,545.321716,238.468778,345.239192,474.928590,79.232918
1458,DEG10610,238.358,251.284424,416.575592,223.700226,580.999817,328.082976,416.871818,500.173148,52.944775


## Create CAMELS-DE topographic attributes csv

Rename columns to be consistent with CAMELS-GB, CAMELS-CH

In [4]:
# dataframe to store results
df_results = pd.DataFrame(index=camels_ids)

for camels_id in camels_ids:
    # add provider_id
    provider_id = meta[meta["camels_id"] == camels_id]["provider_id"].values[0]
    df_results.loc[camels_id, "provider_id"] = provider_id

    # add gauge_name
    gauge_name = meta[meta["camels_id"] == camels_id]["gauge_name"].values[0]
    df_results.loc[camels_id, "gauge_name"] = gauge_name

    # add water_body_name
    water_body_name = meta[meta["camels_id"] == camels_id]["waterbody_name"].values[0]
    df_results.loc[camels_id, "water_body_name"] = water_body_name

    # add federal_state
    federal_state = meta[meta["camels_id"] == camels_id]["federal_state"].values[0]
    df_results.loc[camels_id, "federal_state"] = federal_state

    # add latitude and longitude
    gauge_lat = meta[meta["camels_id"] == camels_id]["lat"].values[0]
    gauge_lon = meta[meta["camels_id"] == camels_id]["lon"].values[0]
    df_results.loc[camels_id, "gauge_lat"] = round(gauge_lat, 6) # round to 6 decimal places to have a precision of ~0.11m
    df_results.loc[camels_id, "gauge_lon"] = round(gauge_lon, 6) # round to 6 decimal places to have a precision of ~0.11m

    # add easting and northing as integers
    gauge_easting = meta[meta["camels_id"] == camels_id]["x"].values[0]
    gauge_northing = meta[meta["camels_id"] == camels_id]["y"].values[0]
    df_results.loc[camels_id, "gauge_easting"] = int(gauge_easting)
    df_results.loc[camels_id, "gauge_northing"] = int(gauge_northing)

    # add gauge elevation from metadata
    gauge_elev_metadata = meta[meta["camels_id"] == camels_id]["gauge_elevation"].values[0]
    df_results.loc[camels_id, "gauge_elev_metadata"] = round(gauge_elev_metadata, 2)

    # add area from metadata
    area_metadata = meta[meta["camels_id"] == camels_id]["area"].values[0]
    df_results.loc[camels_id, "area_metadata"] = round(area_metadata, 2)

    # add gauge elevation from dem
    gauge_elev = dem[meta["camels_id"] == camels_id]["gauge_elevation_from_dem"].values[0]
    df_results.loc[camels_id, "gauge_elev"] = round(gauge_elev, 2)

    # add merit hydro area
    s = Station(camels_id)
    area = (s.get_catchment("merit_hydro").to_crs("EPSG:6933").area / 1e6)[0]
    df_results.loc[camels_id, "area"] = round(area, 2)

    # add catchment elevation statistics
    elev_mean = dem[meta["camels_id"] == camels_id]["mean_catchment_elevation_from_dem"].values[0]
    elev_min = dem[meta["camels_id"] == camels_id]["min_catchment_elevation_from_dem"].values[0]
    elev_5 = dem[meta["camels_id"] == camels_id]["quantile0.05_catchment_elevation_from_dem"].values[0] 
    elev_50 = dem[meta["camels_id"] == camels_id]["quantile0.5_catchment_elevation_from_dem"].values[0]
    elev_95 = dem[meta["camels_id"] == camels_id]["quantile0.95_catchment_elevation_from_dem"].values[0]
    elev_max = dem[meta["camels_id"] == camels_id]["max_catchment_elevation_from_dem"].values[0]
    df_results.loc[camels_id, "elev_mean"] = round(elev_mean, 2)
    df_results.loc[camels_id, "elev_min"] = round(elev_min, 2)
    df_results.loc[camels_id, "elev_5"] = round(elev_5, 2)
    df_results.loc[camels_id, "elev_50"] = round(elev_50, 2)
    df_results.loc[camels_id, "elev_95"] = round(elev_95, 2)
    df_results.loc[camels_id, "elev_max"] = round(elev_max, 2)

df_results

100%|██████████| 1460/1460 [00:44<00:00, 32.89it/s]


,provider_id,gauge_name,water_body_name,federal_state,gauge_lat,gauge_lon,gauge_easting,gauge_northing,gauge_elev_metadata,area_metadata,gauge_elev,area,elev_mean,elev_min,elev_5,elev_50,elev_95,elev_max
DE110000,105,Kirchen-Hausen,Donau,Baden-Württemberg,47.93,8.68,4222273.0,2757760.0,657.33,758.53,661.99,763.07,822.25,658.53,681.92,792.26,1037.07,1159.79
DE110010,106,Möhringen,Donau,Baden-Württemberg,47.95,8.76,4228335.0,2760262.0,649.16,826.96,650.84,831.29,820.25,649.94,680.98,792.74,1033.22,1159.79
DE110020,120,Hundersingen,Donau,Baden-Württemberg,48.07,9.40,4275974.0,2773394.0,542.53,2621.32,546.44,2618.60,774.90,541.59,621.95,765.67,962.75,1159.79
DE110030,125,Berg,Donau,Baden-Württemberg,48.27,9.73,4301054.0,2794786.0,489.90,4072.79,493.75,4093.96,729.47,488.79,555.29,726.97,933.64,1159.79
DE110040,129,Achstetten,Baierzer Rot,Baden-Württemberg,48.26,9.90,4313632.0,2794440.0,489.32,264.39,491.35,272.81,628.93,490.26,524.48,626.60,733.95,791.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DEG10580,427010,Unterbreizbach-Räsa,Ulster,Thüringen,50.81,9.98,4319357.0,3077271.0,233.32,399.00,236.30,402.07,486.59,234.86,293.90,459.05,775.27,942.45
DEG10590,420120,Vacha,Werra,Thüringen,50.83,10.05,4324358.0,3080274.0,222.68,2246.00,225.59,2245.18,473.41,223.51,282.77,451.89,736.47,988.19
DEG10600,575110,Wasserthaleben,Helbe,Thüringen,51.26,10.89,4383230.0,3127720.0,174.32,374.30,176.61,372.43,350.21,176.31,238.47,345.24,474.93,545.32
DEG10610,577320,Weida,Weida,Thüringen,50.76,12.06,4466470.0,3074266.0,238.36,296.70,251.28,297.07,416.58,223.70,328.08,416.87,500.17,581.00


## Save results

In [8]:
df_results.to_csv("../output_data/camels_de/CAMELS_DE_topographic_attributes.csv", index_label="gauge_id")